# PRACTICA GUIADA 1: Agregando y agrupando datos

* La sumarización es parte fundamental del análisis: computar medidas como ``sum()``, ``mean()``, ``median()``, ``min()`` y ``max()`` son tareas básicas para comenzar a explorar un dataset.

* En esta sección veremos algunas opciones para realizar agregaciones de datos en Pandas.


In [1]:
import numpy as np
import pandas as pd

## Analizando la encuesta en centros de atención turística

El GCBA realiza encuestas a los turistas que se acercan a los centros de atención. Se pregunta el motivo de la consulta, los días que dura el viaje, el país de origen, entre otras cosas.

El dataset es de acceso público en el portal de datos abiertos del GCBA.


In [2]:
!ls notebooks.azure.com/fabianalejandrogomez/projects/digital-house-data-science/html/Clase-03/resultado-de-encuestas-2016.csv

ls: cannot access 'notebooks.azure.com/fabianalejandrogomez/projects/digital-house-data-science/html/Clase-03/resultado-de-encuestas-2016.csv': No such file or directory


In [3]:
pd.read_csv('https://notebooks.azure.com/fabianalejandrogomez/projects/digital-house-data-science/html/Clase-03/resultado-de-encuestas-2016.csv')

HTTPError: HTTP Error 404: Not Found

In [ ]:
turistas = pd.read_csv('resultado-de-encuestas-2016.csv', sep = ";")
turistas.shape

In [ ]:
turistas.head()

Veamos todas las columnas

In [ ]:
turistas.columns

Veamos si el dataset tiene datos faltantes.

In [ ]:
turistas.isnull().sum()

Parecería ser que no hay datos faltantes.

## Agregaciones simples en Pandas

* Al igual que en un array de una dimensión para una ``Series`` la agregación devuelve un valor único. Veamos cuántos días en total y en promedio dura el viaje de las personas que se acercaron a estos centros de atención.

In [ ]:
turistas["PERNOCTACIONES"].sample(25)

Veamos que la columna incluye valores no numéricos. Primero tenemos que removerlos y luego podemos hacer los cálculos.

In [ ]:
# el método str.extract permite extraer un grupo pasando una REGEX. Para probar REGEX se puede entrar en https://regex101.com/ o https://regexr.com/ 

turistas['PERNOCTACIONES'] = turistas['PERNOCTACIONES']\
                             .str.extract('(\d+)', expand=False).astype('float')

In [ ]:
turistas["PERNOCTACIONES"].sample(25)

In [ ]:
turistas["PERNOCTACIONES"].fillna(turistas["PERNOCTACIONES"].mean(), inplace=True)

In [ ]:
print('Suma de Pernoctaciones: {:.2f}'.format(turistas["PERNOCTACIONES"].sum()))

In [ ]:
print('Media de Pernoctaciones: {:.2f}'.format(turistas["PERNOCTACIONES"].mean()))

* Para un ``DataFrame``, los `aggregate` retornan por default resultados al interior de cada columna:

* Especificando el argumento ``axis`` es posible agregar los resultados para cada fila.

**Nota:** Al igual que en el caso de NumPy, en `axis` se define la dimensión sobre la que se colapsan los datos. Si quiero resultados por fila, entonces, `axis='columns'` y en otro caso `axis='rows'`

En el archivo ya viene calculada la cantidad de motivos por los que se acercan los turistas. Calculemos nosotros ese dato y comparémoslo con el original, ya que es común que hayan errores en la información.

In [ ]:
# Los motivos van desde la columna ACCESIBILIDAD hasta USO_DE_INSTALACIONES
# Calculo la suma por fila
n_motivos_calculado = turistas.loc[:,"ACCESIBILIDAD":"USO_DE_INSTALACIONES"]\
                      .sum(axis = 'columns')
# Me quedo con el valor original
n_motivos_original = turistas['CANT_DE_MOTIVOS']

# Vemos que hay errores en el campo original:
print('¿Cantidad total calculada y original son iguales? {}'\
       .format(n_motivos_calculado.equals(n_motivos_original)))

In [ ]:
# ¿Cuántos están mal? 52
(n_motivos_calculado != n_motivos_original).sum()

* Las ``Series`` y ``DataFrame`` de Pandas incluyen muchas funciones de agregación.

* Hay, además, un método ``describe()`` que computa algunas medidas agregadas por defecto para cada columna y devuelve un `DataFrame` como resultado.


In [ ]:
turistas[['PASAJEROS', 'PERNOCTACIONES']].describe()

Podemos observar que hay valores raros, ya que en pasajeros vemos 150... ¿es un error o nos falta conocer "el negocio"? Tal vez alguien consulto por un grupo de viajeros...

* ¿Qué pueden decir de estos datos?

In [ ]:
# Alternativas para el método .describe()

turistas.describe(percentiles=[0.1,0.25,0.35,0.5,0.8], include='float')

Resumen de algunas funciones de agregación en Pandas:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

Todos son métodos de los objetos ``DataFrame`` y ``Series``.

* Muchas veces esto no es suficiente y es necesario usar otras operaciones de sumarización.
* La operación ``groupby`` permite computar medidas agregadas de forma eficente en subsets de los datos.

## GroupBy: Split, Apply, Combine

* Muchas veces es importante poder realizar operaciones de agregación de forma condicional a algún subconjunto de datos (por ejemplo, para los casos que cumplen alguna condición). Esto es implementado por el operador `groupby`
* El nombre `groupby` proviene del lenguaje SQL.
* Es útil pensarlo en los términos de Hadley Wickham: *split, apply, combine* (dividir-aplicar-combinar)

### Split, apply, combine

![title](img/split-apply-combine.png)

- El paso *split*  implica dividir y reagrupar un ``DataFrame`` en base a una determinada key.
- El paso *apply* supone computar alguna función (generalmente, alguna agregación, transformación o filtro) sobre los grupos constituidos en el paso anterior.
- El paso *combine* hace un "merge" de los resultados de dichas operaciones en un nuevo array.

* Si bien cada uno de los pasos pueden hacerse "manualmente", el ``GroupBy`` generalmente puede hacerlo en un solo paso.
* La ventaja del ``GroupBy`` es que permite abstraer los tres pasos anteriores: el usuario no necesita pensar en "cómo" hacer el cómputo, sino más bien pensar la operación como un todo.

* La operación split-apply-combine más básica con el método ``groupby()`` es pasar el nombre de una determinada clave de columna:

### El objeto GroupBy

* El objeto ``GroupBy`` es un abstracción muy flexible.
* Puede ser tratado como una colección de `DataFrame` y hace algunas cosas complicadas por detrás...
* Quizá las operaciones más importantes de un objeto ``GroupBy`` sean las *aggregate*, *filter*, *transform*, y *apply*

In [ ]:
turistas.groupby('PAIS_EXT')

In [ ]:
turistas.groupby('PAIS_EXT').groups

* Notar que no se retorna un ``DataFrame`` sino un objeto ``DataFrameGroupBy``.
* Se puede pensar en el `DataFrameGroupBy` como una vista especial de un `DataFrame` que construye los grupos pero que no realiza ningún cómputo hasta que la etapa de agregación es aplicada.
* Esta **"lazy evaluation"** hace que las operaciones de agregación puedan ser computadas de forma muy eficiente.
* Para producir un resultado podemos aplicar una función de agregación a este ``DataFrameGroupBy`` que a realizar la operación (apply) y avanzar en el paso de combine.

In [ ]:
turistas[['PASAJEROS', 'PERNOCTACIONES','PAIS_EXT']]\
                                      .groupby('PAIS_EXT').sum().sort_values("PASAJEROS",\
                                       ascending = False).head()

* Se podría realizar virtualmente cualquier operación común de agregación de Pandas o NumPy y casi cualquier operación válida para un `DataFrame` como veremos a continuación.

#### Indexing de columnas

* El objeto ``GroupBy`` soporta el indexado de columnas de la misma forma que un ``DataFrame``, y devuelve un objeto ``GroupBy`` modificado:

In [ ]:
turistas.groupby('PAIS_EXT')['ALREDEDORES_BA']

* Seleccionamos una ``Series`` particular del  ``DataFrame`` original referenciándolo con su nombre de columna.
* Al igual que antes, no se realiza ningún cómputo hasta que llamamos alguna función de agregación sobre el objeto.

In [ ]:
turistas.groupby('PAIS_EXT')['ALREDEDORES_BA'].sum().sort_values(ascending = False).head()

* ¿Qué se observa en estos datos?

#### Iteración sobre grupos

* El objeto ``GroupBy`` soporta la iteración directa sobre grupos, devolviendo cada grupo como una ``Series`` o un ``DataFrame``.


In [ ]:
for (pais, group) in turistas.groupby('PAIS_EXT'):
    print("{0:20} shape={1} ".format(pais, group.shape))

* Si bien podemos hacer este tipo de operaciones de forma manual, veremos enseguida la potencialidad que tienen las funcionalidades ``apply``.

#### Dispatch methods

* Cualquier método no implementado de forma explícita por el objeto ``GroupBy`` será ejecutado en cada grupo. Estos métodos se heredan de la clase DataFrame. 
* Por ejemplo, se puede usar el método ``describe()`` de ``DataFrame``s para realizar muchas operaciones de agregación al interior de cada grupo.

In [ ]:
turistas.groupby('PAIS_EXT')[['PASAJEROS', 'PERNOCTACIONES']].describe()

* ¿Qué puede decirse de estos datos? 
* Este es solo un ejemplo de la utilidad de estos métodos. Notar que son aplicados a cada uno de los grupos individuales y que los resultados son combinados en un objeto `GroupBy` y retornados.

### Aggregate, filter, transform, apply

* Hay muchas más opciones de operaciones disponibles.
* Los objetos ``GroupBy`` tienen algunos métodos muy útiles: ``aggregate()``, ``filter()``, ``transform()`` y ``apply()`` que implementan muchas operaciones en la etapa previa al "combine"

* Ilustremos estas operaciones con el siguiente ``DataFrame``:

#### Aggregation

* El método ``aggregate()`` permite una gran flexibilidad:
* Puede tomar un string, una función o una lista y computar todos los agregados en un solo paso.

In [ ]:
turistas.groupby('ARGENTINA')["PASAJEROS"].aggregate([np.min, np.mean, np.median,\
                                                      np.max, np.sum])

* Otra forma útil es pasar un diccionario que mapea nombres de columnas con operaciones. De esta forma se puede espeficiar una operación distinta a cada columna.

In [ ]:
turistas.groupby('ARGENTINA')['PASAJEROS'].aggregate([np.min, 
                 np.median, np.max, np.sum]).rename(columns={'amin': 'grupo_mas_chico',
                'median': 'grupo_mediano','amax':'grupo_mas_chico','sum':'total'}).head()

#### Filtering

* Una operación filtering permite "descartar" datos basado en propiedades del grupo.
* Por ejemplo, podríamos querer mantener todos los grupos en los que la desviación estándar sea mayor que algún valor de corte:

In [ ]:
def filter_func(x, lim):
    return x["PERNOCTACIONES"].std() > lim

In [ ]:
filter_func(turistas, 20)

In [ ]:
turistas.groupby('ARGENTINA').filter(filter_func, lim=20)["ARGENTINA"].unique()

* La función de filtro retorna un booleano especificando si el grupo pasa o no el filtro. 

#### Transformation

* Mientras que aggregation devuelve una versión reducida de los datos, transformation retorna alguna versión transformada de los datos para, luego, hacer el combine.
* El output de una transformation es del mismo `shape` que el input.

In [ ]:
def center_mean(x):
    return(x - x.mean())

print(turistas[["ARGENTINA","PASAJEROS"]].groupby('ARGENTINA').transform(center_mean))

* ¿Qué hace el siguiente bloque de código?

#### El método `apply()`

* El método `apply()` permite aplicar alguna función dada a los resultados del group.
* La función debería tomar como input ``DataFrame`` y devolver un objeto de Pandas (``DataFrame``, ``Series``) o un escalar. 
* La operación combine se adaptará al tipo de salida devuelta.

In [ ]:
# El resultado es una columna PASAJEROS normalizada por la participación 
#del grupo en el total de pasajeros de la provincia:

def norm_by_data2(x):
    # x is a DataFrame of group values
    x['PASAJEROS'] /= x['PERNOCTACIONES'].sum()
    return x

turistas[["ARGENTINA","PASAJEROS","PERNOCTACIONES"]].groupby('ARGENTINA')\
                                           .apply(norm_by_data2).sample(15)


* ¿Qué operación produce el bloque de código anterior?

### Especificando la clave del "split"

* En el ejemplo anterior se hacía el split del ``DataFrame`` sobre una sola columna.
* Hay otras opciones...

#### Una lista, array, series, o index que contiene las claves del grouping 

In [ ]:
L = ["CAT","ARGENTINA"]
turistas.groupby(L).sum()

In [ ]:
turistas.groupby("ARGENTINA").sum().head()


In [ ]:
# Es totalmente equivaliente a
turistas.groupby(turistas["ARGENTINA"]).sum().head()


#### Un dict o serie mapeando index a grupos

In [ ]:
turistas2 = turistas.set_index('ARGENTINA')
mapping = {'CHACO': 'C', 'CHUBUT': 'D', 'CATAMARCA': 'E'}
turistas2.groupby(mapping).sum()

### Ejemplo de aplicación

* Imaginemos que nos consultan cuáles son los lugares de CABA más visitados por los turistas argentinos, según su procedencia. Para ello suponemos que el CAT al que van los turistas indica que fueron a visitar ese lugar. 
* Entonces primero filtramos los turistas extranjeros, agrupamos por provincia, llenamos los Na con 0 y dividimos todo por la cantidad total de turistas argentinos. De esta manera, lo representamos como proporción del total

In [ ]:
turistas.loc[turistas.ARGENTINA != " ",].groupby(['ARGENTINA', 
                      'CAT'])['PASAJEROS'].sum().unstack(fill_value=0)/sum(turistas.loc[turistas.ARGENTINA != " ",
                                                                                            "PASAJEROS"])*100